In [1]:
import time
import cv2
import os
import torch
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
from torchvision.transforms import Compose
from depth_anything.dpt import DepthAnything
from depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet

In [2]:
img_path = '/Users/ewojcik/Code/ai/AIxperiments/Images/Depth Estimation/imgs'
outdir = './output'
encoder = 'vitl'  # choices: 'vits', 'vitb', 'vitl'
pred_only = False
grayscale = False

margin_width = 50
caption_height = 60

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_thickness = 2

DEVICE = 'mps'

In [3]:
if os.path.isfile(img_path):
    if img_path.endswith('txt'):
        with open(img_path, 'r') as f:
            filenames = f.read().splitlines()
    else:
        filenames = [img_path]
else:
    filenames = os.listdir(img_path)
    filenames = [os.path.join(img_path, filename) for filename in filenames if not filename.startswith('.')]
    filenames.sort()

os.makedirs(outdir, exist_ok=True)

In [4]:
depth_anything = DepthAnything.from_pretrained('LiheYoung/depth_anything_{}14'.format(encoder)).to(DEVICE).eval()

total_params = sum(param.numel() for param in depth_anything.parameters())
print('Total parameters: {:.2f}M'.format(total_params / 1e6))

transform = Compose([
    Resize(
        width=518,
        height=518,
        resize_target=False,
        keep_aspect_ratio=True,
        ensure_multiple_of=14,
        resize_method='lower_bound',
        image_interpolation_method=cv2.INTER_CUBIC,
    ),
    NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    PrepareForNet(),
])

xFormers not available
xFormers not available


Total parameters: 335.32M


In [5]:
for filename in tqdm(filenames):

    start_time = time.time()

    raw_image = cv2.imread(filename)
    image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB) / 255.0
    
    h, w = image.shape[:2]
    
    image = transform({'image': image})['image']
    image = torch.from_numpy(image).unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        depth = depth_anything(image)
    
    depth = F.interpolate(depth[None], (h, w), mode='bilinear', align_corners=False)[0, 0]
    depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
    
    depth = depth.cpu().numpy().astype(np.uint8)
    
    if grayscale:
        depth = np.repeat(depth[..., np.newaxis], 3, axis=-1)
    else:
        depth = cv2.applyColorMap(depth, cv2.COLORMAP_INFERNO)
    
    filename = os.path.basename(filename)
    
    if pred_only:
        cv2.imwrite(os.path.join(outdir, filename[:filename.rfind('.')] + '_depth.png'), depth)
    else:
        split_region = np.ones((raw_image.shape[0], margin_width, 3), dtype=np.uint8) * 255
        combined_results = cv2.hconcat([raw_image, split_region, depth])
        
        caption_space = np.ones((caption_height, combined_results.shape[1], 3), dtype=np.uint8) * 255
        captions = ['Raw image', 'Depth Anything']
        segment_width = w + margin_width
        
        for i, caption in enumerate(captions):
            # Calculate text size
            text_size = cv2.getTextSize(caption, font, font_scale, font_thickness)[0]

            # Calculate x-coordinate to center the text
            text_x = int((segment_width * i) + (w - text_size[0]) / 2)

            # Add text caption
            cv2.putText(caption_space, caption, (text_x, 40), font, font_scale, (0, 0, 0), font_thickness)
        
        final_result = cv2.vconcat([caption_space, combined_results])
        
        cv2.imwrite(os.path.join(outdir, filename[:filename.rfind('.')] + '_img_depth.png'), final_result)
    end_time = time.time()  # End timing
    execution_time = end_time - start_time
    print(f"\tProcessing time for {filename}: {execution_time:.4f} seconds")

100%|██████████| 1/1 [00:03<00:00,  3.56s/it]

	Processing time for avatar.jpeg: 3.5608 seconds
